# Symbolic Brusselator PDE

http://methodoflines.sciml.ai/dev/tutorials/brusselator/#Using-the-Brusselator-PDE-as-an-example

The Brusselator PDE is defined as follows:

$$
\begin{align}
\frac{\partial u}{\partial t} &= 1 + u^2v - 4.4u + \alpha (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}) + f(x, y, t) \\
\frac{\partial v}{\partial t} &= 3.4u - u^2 v + \alpha (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2})
\end{align}
$$

where

$$
f(x, y, t) = 
\begin{cases}
5 \qquad \text{if} (x - 0.3)^2 + (y - 0.6)^2 \leq 0.1^2 \ and \  t \geq 1.1  \\
0 \qquad \text{otherwise}
\end{cases}
$$

and the initial conditions are

$$
\begin{align}
u(x, y, 0) &= 22(y(1-y))^{1.5} \\
v(x, y, 0) &= 27(x(1-x))^{1.5}
\end{align}
$$

with the periodic boundary condition

$$
\begin{align}
u(x+1, y, 0) &= u(x, y, t)  \\
u(x, y+1, 0) &= u(x, y, t)
\end{align}
$$

on a timespan of $t \in [0, 11.5]$.

In [1]:
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using DomainSets
using Plots

In [2]:
@parameters x y t
@variables u(..) v(..)

Dt = Differential(t)
Dx = Differential(x)
Dy = Differential(y)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

# Laplacian
∇²(u) = Dxx(u) + Dyy(u)

∇² (generic function with 1 method)

In [3]:
brusselator_f(x, y, t) = (((x-0.3)^2 + (y-0.6)^2) <= 0.1^2) * (t >= 1.1) * 5

brusselator_f (generic function with 1 method)

In [4]:
x_min = y_min = t_min = 0.0
x_max = y_max = 1.0
t_max = 11.5
α = 10.0

u0(x, y, t) = 22( y * ( 1 - y ))^(3/2)
v0(x, y, t) = 27( x * ( 1 - x ))^(3/2)

eq = [Dt(u(x,y,t)) ~ 1. + v(x,y,t)*u(x,y,t)^2 - 4.4*u(x,y,t) + α*∇²(u(x,y,t)) + brusselator_f(x, y, t),
      Dt(v(x,y,t)) ~ 3.4*u(x,y,t) - v(x,y,t)*u(x,y,t)^2 + α*∇²(v(x,y,t))]

2-element Vector{Equation}:
 Differential(t)(u(x, y, t)) ~ 1.0 + 10.0Differential(x)(Differential(x)(u(x, y, t))) + 10.0Differential(y)(Differential(y)(u(x, y, t))) + (u(x, y, t)^2)*v(x, y, t) + 5(t >= 1.1)*(((x - 0.3)^2 + (y - 0.6)^2) <= 0.010000000000000002) - 4.4u(x, y, t)
 Differential(t)(v(x, y, t)) ~ 10.0Differential(x)(Differential(x)(v(x, y, t))) + 10.0Differential(y)(Differential(y)(v(x, y, t))) + 3.4u(x, y, t) - (u(x, y, t)^2)*v(x, y, t)

In [5]:
domains = [x ∈ Interval(x_min, x_max),
           y ∈ Interval(y_min, y_max),
           t ∈ Interval(t_min, t_max)]

3-element Vector{Symbolics.VarDomainPairing}:
 Symbolics.VarDomainPairing(x, 0.0..1.0)
 Symbolics.VarDomainPairing(y, 0.0..1.0)
 Symbolics.VarDomainPairing(t, 0.0..11.5)

In [6]:
# Periodic BCs
bcs = [u(x, y, 0) ~ u0(x, y, 0),
       u(0, y, t) ~ u(1, y, t),
       u(x, 0, t) ~ u(x, 1, t),

       v(x, y , 0) ~ v0(x, y, 0),
       v(0, y , t) ~ v(1, y, t),
       v(x, 0 , t) ~ v(x, 1, t)] 

6-element Vector{Equation}:
 u(x, y, 0) ~ 22.0(y^1.5)*((1 - y)^1.5)
 u(0, y, t) ~ u(1, y, t)
 u(x, 0, t) ~ u(x, 1, t)
 v(x, y, 0) ~ 27.0(x^1.5)*((1 - x)^1.5)
 v(0, y, t) ~ v(1, y, t)
 v(x, 0, t) ~ v(x, 1, t)

In [7]:
@named pdesys = PDESystem(eq, bcs, domains, [x, y, t], [u(x,y,t), v(x,y,t)])

PDESystem
Equations: Equation[Differential(t)(u(x, y, t)) ~ 1.0 + 10.0Differential(x)(Differential(x)(u(x, y, t))) + 10.0Differential(y)(Differential(y)(u(x, y, t))) + (u(x, y, t)^2)*v(x, y, t) + 5(t >= 1.1)*(((x - 0.3)^2 + (y - 0.6)^2) <= 0.010000000000000002) - 4.4u(x, y, t), Differential(t)(v(x, y, t)) ~ 10.0Differential(x)(Differential(x)(v(x, y, t))) + 10.0Differential(y)(Differential(y)(v(x, y, t))) + 3.4u(x, y, t) - (u(x, y, t)^2)*v(x, y, t)]
Boundary Conditions: Equation[u(x, y, 0) ~ 22.0(y^1.5)*((1 - y)^1.5), u(0, y, t) ~ u(1, y, t), u(x, 0, t) ~ u(x, 1, t), v(x, y, 0) ~ 27.0(x^1.5)*((1 - x)^1.5), v(0, y, t) ~ v(1, y, t), v(x, 0, t) ~ v(x, 1, t)]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(x, 0.0..1.0), Symbolics.VarDomainPairing(y, 0.0..1.0), Symbolics.VarDomainPairing(t, 0.0..11.5)]
Dependent Variables: Num[u(x, y, t), v(x, y, t)]
Independent Variables: Num[x, y, t]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

In [11]:
# Discretization
N = 10

dx = (x_max - x_min) / N
dy = (y_max - y_min) / N

order = 2

discretization = MOLFiniteDifference([x=>dx, y=>dy], t, approx_order=order, grid_align=center_align)

MOLFiniteDifference{MethodOfLines.CenterAlignedGrid}(Pair{Num, Float64}[x => 0.1, y => 0.1], t, 2, UpwindScheme(1), MethodOfLines.CenterAlignedGrid(), Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())

In [12]:
# Discretize the PDESystem to an ODEProblem.
prob = discretize(pdesys, discretization)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 11.5)
u0: 200-element Vector{Float64}:
 0.5940000000000001
 0.5940000000000001
 0.5940000000000001
 0.5940000000000001
 0.5940000000000001
 0.5940000000000001
 0.5940000000000001
 0.5940000000000001
 0.5940000000000001
 0.5940000000000001
 ⋮
 1.7280000000000006
 2.5983204190399607
 3.174538706646999
 3.3750000000000004
 3.174538706646999
 2.5983204190399616
 1.7279999999999998
 0.7289999999999999
 0.0

In [13]:
# Solvers: https://diffeq.sciml.ai/stable/solvers/ode_solve/
sol = solve(prob, TRBDF2(), saveat=0.1)

MethodError: MethodError: no method matching interp_summary(::Dict{Num, Interpolations.GriddedInterpolation{Float64, 3, Array{Float64, 3}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}}}})
Closest candidates are:
  interp_summary(!Matched::Union{DESolution, SciMLSolution}) at ~/.julia/packages/SciMLBase/idlaI/src/interpolation.jl:42
  interp_summary(!Matched::OrdinaryDiffEq.OrdinaryDiffEqInterpolation{cacheType}) where cacheType at ~/.julia/packages/OrdinaryDiffEq/nidTv/src/interp_func.jl:25
  interp_summary(!Matched::SciMLBase.HermiteInterpolation) at ~/.julia/packages/SciMLBase/idlaI/src/interpolation.jl:35
  ...

In [ ]:
discrete_x = sol[x]
discrete_t = sol[y]
discrete_t = sol[t]

solu = sol[u(x, y, t)]
solv = sol[v(x, y, t)] 

In [ ]:
umax = maximum(maximum, solu)
vmax = maximum(maximum, solu)

In [ ]:
using Plots

anim = @animate for k in 1:length(discrete_t)
    heatmap(solu[k, 2:end, 2:end], title="u @ t=$(discrete_t[k])", clims = (0.0, 4.5)) # 2:end since end = 1, periodic condition
end

mp4(anim, fps = 8)

In [ ]:
anim = @animate for k in 1:length(discrete_t)
    heatmap(solv[k, 2:end, 2:end], title="v @ t=$(discrete_t[k])", clims = (0.0, 4.5))
end

mp4(anim, fps = 8)

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()